# 📚 Project: Books to Scrape Data Pipeline

**Author:** Hanna Kovalenko
**Goal:** Scrape, process, and clean e-commerce data to analyze book prices and ratings across different genres.

## ⚙️ Workflow
This notebook executes an ETL (Extract, Transform, Load) process:

1.  **Extract:**
    *   Connects to `books.toscrape.com`.
    *   Extracts a list of all book categories (Genres) and their URLs.
    *   Iterates through each genre to collect individual book data (Title, Price, Rating, Availability).
2.  **Transform:**
    *   Extracts full titles from HTML attributes (handling truncated text).
    *   Converts string ratings (e.g., "Three") into integers (3).
    *   Cleans price strings (removes currency symbols and non-numeric characters).
3.  **Load:**
    *   Compiles data into a Pandas DataFrame.
    *   Exports the clean dataset to CSV.

## 🛠 Libraries
*   `requests`: For HTTP requests.
*   `BeautifulSoup`: For parsing HTML.
*   `pandas`: For data structuring and cleaning.
*   `time`: For polite scraping delays.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [5]:
url = 'http://books.toscrape.com/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
# Look for jenre panel 
sidebar = soup.find('div', class_='side_categories')

In [8]:
# Look for catagories 
categories = sidebar.find_all('a')[1:]

In [9]:
genre_links = []

for cat in categories:
    # Pull jenre names
    genre_name = cat.text.strip()
    
    # Pull link (href)
    # Concat common link and link to book jenre
    full_link = 'http://books.toscrape.com/' + cat['href']
    
    genre_links.append({
        'Genre': genre_name,
        'URL': full_link
    })

In [10]:
df_genres = pd.DataFrame(genre_links)
print(f"Jenres finded: {len(df_genres)}")
df_genres.head()

Найдено жанров: 50


,Genre,URL
0,Travel,http://books.toscrape.com/catalogue/category/b...
1,Mystery,http://books.toscrape.com/catalogue/category/b...
2,Historical Fiction,http://books.toscrape.com/catalogue/category/b...
3,Sequential Art,http://books.toscrape.com/catalogue/category/b...
4,Classics,http://books.toscrape.com/catalogue/category/b...


In [13]:
import time

books_data = []

print("Start collecting books by jenres")

for index, row in df_genres.iterrows():
    current_genre = row['Genre']
    current_url = row['URL']
    
    print(f"📂 Enter in category: {current_genre}...")
    
    try:
        response = requests.get(current_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Look for all books on the page
        all_books = soup.find_all('article', class_='product_pod')
        
        for book in all_books:
            # 1. Name (take full name from title)
            title = book.find('h3').find('a')['title']
            
            # 2. Price
            price = book.find('p', class_='price_color').text
            
            # 3. Raiting
            # It is put in class: <p class="star-rating Three">
            # We take only second element (Index 1) with stars count 
            star_rating = book.find('p', class_='star-rating')['class'][1]
            
            # 4. Availability (In stock)
            availability = book.find('p', class_='instock availability').text.strip()
            
            # Save data in the Dataframe
            books_data.append({
                'Genre': current_genre, 
                'Title': title,
                'Price': price,
                'Rating': star_rating,
                'Availability': availability
            })
            
        # Pause
        time.sleep(1)
        
    except Exception as e:
        print(f"⚠️ Error in the jenre {current_genre}: {e}")

print("-" * 30)
print(f"✅ Ready! {len(books_data)} was collected")



Start collecting books by jenres
📂 Enter in catagory: Travel...
📂 Enter in catagory: Mystery...
📂 Enter in catagory: Historical Fiction...
📂 Enter in catagory: Sequential Art...
📂 Enter in catagory: Classics...
📂 Enter in catagory: Philosophy...
📂 Enter in catagory: Romance...
📂 Enter in catagory: Womens Fiction...
📂 Enter in catagory: Fiction...
📂 Enter in catagory: Childrens...
📂 Enter in catagory: Religion...
📂 Enter in catagory: Nonfiction...
📂 Enter in catagory: Music...
📂 Enter in catagory: Default...
📂 Enter in catagory: Science Fiction...
📂 Enter in catagory: Sports and Games...
📂 Enter in catagory: Add a comment...
📂 Enter in catagory: Fantasy...
📂 Enter in catagory: New Adult...
📂 Enter in catagory: Young Adult...
📂 Enter in catagory: Science...
📂 Enter in catagory: Poetry...
📂 Enter in catagory: Paranormal...
📂 Enter in catagory: Art...
📂 Enter in catagory: Psychology...
📂 Enter in catagory: Autobiography...
📂 Enter in catagory: Parenting...
📂 Enter in catagory: Adult Fictio

In [14]:
# Look at the result

df_books = pd.DataFrame(books_data)
df_books.head()

,Genre,Title,Price,Rating,Availability
0,Travel,It's Only the Himalayas,Â£45.17,Two,In stock
1,Travel,Full Moon over Noahâs Ark: An Odyssey to Mou...,Â£49.43,Four,In stock
2,Travel,See America: A Celebration of Our National Par...,Â£48.87,Three,In stock
3,Travel,Vagabonding: An Uncommon Guide to the Art of L...,Â£36.94,Two,In stock
4,Travel,Under the Tuscan Sun,Â£37.33,Three,In stock


In [15]:
# 1. Clean raiting using map function (Words to numbers)

rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}

df_books['Rating_Num'] = df_books['Rating'].map(rating_map)


In [18]:
# 2. Clean price (remove symbols)

df_books['Price_Clean'] = df_books['Price'].str[2:].astype(float)

# 3. Create final dataframe 
df_final = df_books[['Title', 'Price_Clean', 'Rating_Num', 'Genre', 'Availability']]
df_final.head()

,Title,Price_Clean,Rating_Num,Genre,Availability
0,It's Only the Himalayas,45.17,2,Travel,In stock
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,4,Travel,In stock
2,See America: A Celebration of Our National Par...,48.87,3,Travel,In stock
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,2,Travel,In stock
4,Under the Tuscan Sun,37.33,3,Travel,In stock


In [21]:
# Save to Excel
df_final.to_excel('books_dataset.xlsx', index=False)